# chatbot "Korean Trip Planner"
"**Korean Trip Planner**" sets up a day-by-day trip to Korea

In [18]:
from chatbot.chatbot import Chatbot

In [19]:
import uuid
import time

In [20]:
import csv
import pandas as pd

## Read korea tourist information csv file

In [21]:
def read_tourism_spots(csv_file_path):
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_string = file.read()
    return csv_string

csv_file_path = './data/test_set.csv'

test_set_locations = read_tourism_spots(csv_file_path)

In [22]:
test_set_locations

'POI_ID,POI_NM,CTPRVN_NM,SIGNGU_NM,LEGALDONG_NM,LC_LO,LC_LA\n21086196,청계천매화거리,서울특별시,성동구,용답동,127.0486307,37.56647803\n20842744,감악산먹거리촌,경기도,파주시,적성면,126.9427856,37.941266\n16404013,분당가구거리,경기도,성남시 분당구,금곡동,127.1018181,37.36256067\n21085878,신봉동먹자골목,경기도,용인시 수지구,신봉동,127.0535667,37.33302522\n8292330,누리천문대,경기도,군포시,대야미동,126.91535,37.33029417\n21085951,왕십리맛골목,서울특별시,성동구,행당동,127.0405544,37.55950009\n20907391,명동만화의거리재미로,서울특별시,중구,충무로2가,126.9854872,37.56065454\n17738915,금촌로데오거리,경기도,파주시,금촌동,126.7674551,37.7517887\n7231190,일광해수욕장,부산광역시,기장군,일광읍,129.2341058,35.25960421\n21086190,안산벚꽃길,서울특별시,서대문구,연희동,126.9383509,37.57943033\n16716532,꽃골목,부산광역시,중구,창선동1가,129.0297727,35.0994815\n7232777,오이도해양단지,경기도,시흥시,정왕동,126.6902931,37.34640271\n21086147,추억의6070거리,경기도,안성시,성남동,127.2706234,37.00205352\n21085908,부평리한식마을,경기도,남양주시,진접읍,127.1931631,37.75832911\n21085954,서울중앙시장쌈밥골목,서울특별시,중구,황학동,127.0197856,37.56679597\n21086107,생선구이골목,부산광역시,중구,남포동6가,129.0283406,35.09644416\n6376544,예술거리,서울특별시,마포구,서교동,126.9242558,37.55286406\n1742157

In [23]:
# role - system instance_context
type_role = "You are the best travel planner who can make efficient travel plans. You make plans that are more efficient and readable than anyone else."

# ? - system type_role
instance_context = """
See this desciption:\n{}.\nPlease write based on the description.
You are the best travel planner who can make efficient travel plans.
<p>When responding:</p>
<ul>
    <li>Please provide information on the travel route, tourist destination (POI_NM), and province (CTPRVN_NM).</li>
    <li>When planning the order of my travel destinations, avoid backtracking between cities. For example, It's better to visit places in Seoul, then move on to destinations in Gyeonggi-do, and finally visit attractions in Busan.</li>
    <li>Please fill out three tourist attractions in Korean and English per day.</li>
    <li>Always incorporate emojis when apt. 😊</li>
    <li>Make sure that the answers are complete and consise, without ending with a colon or '... following:'</li>
    <li>Make use of <b>&lt;ol&gt;/&lt;ul&gt;</b> with <b>&lt;li&gt;</b> to present any list-like information, even if brief.</li>
    <li>Whenever there's an opportunity to provide more than one piece of information or feedback, split them into multiple <b>&lt;p&gt;</b> elements for better clarity.</li>
    <li>Always format responses using valid HTML: e.g., <b>&lt;p&gt;</b> for paragraphs, <b>&lt;ul&gt;/&lt;ol&gt;</b> with <b>&lt;li&gt;</b> for lists, and <b>&lt;b&gt;</b> for emphasis or tourist attractions.</li>
    <li>Maintain a nihilistic humorous tone. Keep it brief, but don't sacrifice clarity for brevity.</li>
    <li>Please organize your travel route efficiently considering latitude (LC_LA) longitude (LC_LO) and order the same CTPRVN_NM value.</li>
</ul>
<p>Example response</p>
Day 1:<br />
<ul>
    <li><b>서울숲은행나무길 (Seoul Forest Ginkgo Tree Road)</b> in Seoul</li>
    <li><b>청계천 (Cheonggyecheon Stream)</b> in Seoul</li>
    <li><b>명동거리 (Myeongdong Street)</b> in Seoul</li>
</ul>

Day 2:<br />
<ul>
    <li><b>남산공원남산케이블카 (Namsan Park Namsan Cable Car)</b> in Seoul</li>
    <li><b>북창동먹자골목 (Bukchang-dong Food Alley)</b> in Seoul</li>
    <li><b>한국인삼주마을 (Korean Ginseng Village)</b> in Gyeonggi Province</li>
</ul>

Day 3:<br />
<ul>
    <li><b>안성다문화마을특구 (Anseong Damunhwa Village Special District)</b> in Gyeonggi Province</li>
    <li><b>의정부부대찌개거리 (Uijeongbu Army Stew Alley)</b> in Gyeonggi Province</li>
    <li><b>용문산토속음식마을 (Yongmun Mountain Traditional Food Village)</b> in Gyeonggi Province</li>
</ul>
Enjoy your trip and make wonderful memories in Korea! 😊🇰🇷

Don't forget always using valid HTML tags espacially, avoid backtracking and make efficient movements<br />
""".format(test_set_locations)

# user
# instance_starter = "Make a plan for a trip in Korea."
instance_starter = "As a Korean trip planner, write one very short message to use at the beginning of the conversation in English"

In [24]:

# Amount of instances to be created
number_of_instances = 4

# Change the following to a list of hardcoded instance IDs if you want to use existing users.
user_ids = [str(uuid.uuid4()) for _ in range(number_of_instances)]

c  = 0 # counter for successful requests, don't change
error_c = 0 # counter for failed requests, don't change
for user_id in user_ids:
    bot = Chatbot(
        database_file="database/chatbot.db", 
        type_id="zhfldk82", # Korean Trip Planner
        user_id=user_id,
        type_name="Korean Trip Planner",
        type_role=type_role,
        instance_context=instance_context,
        instance_starter=instance_starter
    )
    try:
        # each bot should have a first message to the user
        print(bot.start())
    except:
        error_c += 1
        continue
    c+=1
    time.sleep(15) #openai seems to produce more errors if we send the requests too fast.
    
print("successful: {}, failed: {}".format(c, error_c))


["Let's plan your exciting journey through Korea! 🇰🇷✈️"]
["Welcome to Korea! 🇰🇷 Let's plan an amazing trip together!"]
["Welcome to Korea! Let's plan an amazing trip together. 🇰🇷✨"]
["Let's plan an unforgettable trip to Korea together! 🇰🇷✈️"]
successful: 4, failed: 0


##### Obtain URLs of all instances of a type

In [28]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('./database/chatbot.db')
cursor = conn.cursor()

# Execute the query to select all values from the user column
cursor.execute('SELECT user FROM chatbot_instances')

# Fetch all results
users_ids = cursor.fetchall()

# Close the connection
conn.close()

# Check if users_ids is not empty before further processing
if users_ids:
    # Extract user IDs from the list of tuples
    users_ids = [user_id[0] for user_id in users_ids]
    print(users_ids)
else:
    print("No user IDs found.")


['32ce8010-83fb-435a-86e8-a7d0291806df', '3353a16a-3c8b-4f8a-80f7-152826b282f6', '8f481601-66ce-4f81-810b-fad14db3dc73', 'b51c6482-ee04-4293-9f70-fb3b4d358aa1', 'd4ae3edb-a913-42ae-bb79-7375955e2bc9']


In [29]:
# pythonanywhere_username = "choesun1zhaw"

type_id = "zhfldk82" # Korean Trip Planner
DEFAULT_URL = "127.0.0.1:5000"

# local url
for user_id in users_ids:
    print("http://{}/{}/{}/chat".format(DEFAULT_URL, type_id, user_id))
print()

# # pythonanywhere url
# for user_id in users_ids:
#     print("https://{}.pythonanywhere.com/{}/{}/chat".format(pythonanywhere_username, type_id, user_id))

http://127.0.0.1:5000/zhfldk82/32ce8010-83fb-435a-86e8-a7d0291806df/chat
http://127.0.0.1:5000/zhfldk82/3353a16a-3c8b-4f8a-80f7-152826b282f6/chat
http://127.0.0.1:5000/zhfldk82/8f481601-66ce-4f81-810b-fad14db3dc73/chat
http://127.0.0.1:5000/zhfldk82/b51c6482-ee04-4293-9f70-fb3b4d358aa1/chat
http://127.0.0.1:5000/zhfldk82/d4ae3edb-a913-42ae-bb79-7375955e2bc9/chat



### the link of survay


https://choesun1zhaw.pythonanywhere.com/zhfldk82/3353a16a-3c8b-4f8a-80f7-152826b282f6/chat  
https://choesun1zhaw.pythonanywhere.com/zhfldk82/8f481601-66ce-4f81-810b-fad14db3dc73/chat  
https://choesun1zhaw.pythonanywhere.com/zhfldk82/b51c6482-ee04-4293-9f70-fb3b4d358aa1/chat  